In [1]:
import os
from pathlib import Path

import torch
from dotenv import load_dotenv

from torchinfo import summary

from ultralytics import YOLO

In [2]:
load_dotenv(".env", override=True)
if Path(".env.local").exists():
  load_dotenv(".env.local", override=True)

Used environment variables:
- `DATASETS_DIRPATH`: Datasets desination directory path.
- `TMP_DIRPATH`: Temporary directory path.

In [3]:
tmp_dirpath = Path(os.environ["TMP_DIRPATH"]).resolve()
datasets_dirpath = Path(os.environ["DATASETS_DIRPATH"]).resolve()

Trainings are started using `screen` tool.

**MEMO**: list running sessions with `screen -ls`.

In [4]:
def run(cmd, screen=False):
    print(cmd)
    if screen:
        print("Start new screen session ...")
        !screen -dmS train {cmd} && sleep 1
    else:
        !{cmd}

In [5]:
def train(task, model, batch_size, **kwargs):
    data = (datasets_dirpath / "origin" / task / "dataset.yaml").as_posix()
    devices = ','.join([str(i) for i in range(torch.cuda.device_count())])
    bs = torch.cuda.device_count() * batch_size
    cmd = f"yolo {task} train \
data={data} \
model={model} \
batch={bs} \
device={devices}"
    for key, value in kwargs.items():
        cmd += f" {key}={value}"
    run(cmd, screen=True)

In [6]:
def validate(task, model, batch_size, screen=False, **kwargs):
    data = (datasets_dirpath / "origin" / task / "dataset.yaml").as_posix()
    devices = ','.join([str(i) for i in range(torch.cuda.device_count())])
    bs = torch.cuda.device_count() * batch_size
    cmd = f"yolo {task} val \
data={data} \
model={model} \
batch={bs} \
device={devices} \
plots=True"
    for key, value in kwargs.items():
        cmd += f" {key}={value}"
    run(cmd, screen=screen)

In [7]:
def predict(task, model, source="./samples/LFMN_rjWG3WRM5yk_42.png", screen=False, **kwargs):
    data = (datasets_dirpath / "origin" / task / "dataset.yaml").as_posix()
    devices = ','.join([str(i) for i in range(torch.cuda.device_count())])
    cmd = f"yolo {task} predict \
data={data} \
model={model} \
device={devices} \
source={source} \
save=True \
hide_labels=True"
    for key, value in kwargs.items():
        cmd += f" {key}={value}"
    run(cmd, screen=screen)

In [8]:
def info(model_filepath) -> str:
    model = YOLO(model_filepath.as_posix())
    return summary(model.model, input_size=[1, 3, 2048, 2048], col_names=[ "input_size",
                "output_size",
                "num_params",
                "params_percent",
                "kernel_size",
                "mult_adds",
                "trainable"], col_width=20, row_settings=["var_names"])

In [9]:
def export_onnx(model):
    cmd = f"yolo export model={model} format=onnx simplify=True"
    run(cmd)

# Pose

## yolov8n

### From pretrained

In [16]:
train("pose", "yolov8n-pose.pt", batch_size=16, imgsz=2048, epochs=100, workers=8, patience=10, optimizer="AdamW", lr0=0.001, cos_lr=True)

yolo pose train data=/aip/datasets/deel/processed/origin/pose/dataset.yaml model=yolov8n-pose.pt batch=128 device=0,1,2,3,4,5,6,7 imgsz=2048 epochs=100 workers=8 patience=10 optimizer=AdamW lr0=0.001 cos_lr=True
Start new screen session ...


**Copy paste the run directory (ie `./runs/pose/train`) as `./nn/origin/pose/yolov8n/pretrained`.**

In [26]:
model_filepath = Path("./nn/origin/pose/yolov8n/pretrained/weights/best.pt")
assert model_filepath.exists()
info(model_filepath)

/home/aws_install/miniconda3/envs/yolov8/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/aws_install/miniconda3/envs/yolov8/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type (var_name))                            Input Shape          Output Shape         Param #              Param %              Kernel Shape         Mult-Adds            Trainable
PoseModel (PoseModel)                              [1, 3, 2048, 2048]   [1, 13, 86016]       --                        --              --                   --                   False
├─Sequential (model)                               --                   --                   --                   -35.75%              --                   --                   False
│    └─Conv (0)                                    [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  --                        --              --                   --                   False
│    │    └─Conv2d (conv)                          [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  (432)                  0.01%              [3, 3]               452,984,832          False
│    │    └─BatchNorm2d (bn)                       [1, 16, 1024, 1024]  [1, 16, 1

In [27]:
validate("pose", model_filepath.as_posix(), screen=False, batch_size=8, imgsz=2048)

yolo pose val data=/aip/datasets/deel/processed/origin/pose/dataset.yaml model=nn/origin/pose/yolov8n/pretrained/weights/best.pt batch=64 device=0,1,2,3,4,5,6,7 plots=True imgsz=2048
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                                      CUDA:7 (NVIDIA A10G, 22564MiB)
YOLOv8n-pose summary (fused): 187 layers, 3077771 parameters, 0 gradients, 8.3 GFLOPs
val: Scanning /aip/datasets/deel/processed/origin/po

In [28]:
predict("pose", model_filepath.as_posix(), imgsz=2048)

yolo pose predict data=/aip/datasets/deel/processed/origin/pose/dataset.yaml model=nn/origin/pose/yolov8n/pretrained/weights/best.pt device=0,1,2,3,4,5,6,7 source=./samples/LFMN_rjWG3WRM5yk_42.png save=True hide_labels=True imgsz=2048
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                          

Export in ONNX:

In [29]:
export_onnx(model_filepath.as_posix())

yolo export model=nn/origin/pose/yolov8n/pretrained/weights/best.pt format=onnx simplify=True
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CPU
YOLOv8n-pose summary (fused): 187 layers, 3077771 parameters, 0 gradients, 8.3 GFLOPs

PyTorch: starting from nn/origin/pose/yolov8n/pretrained/weights/best.pt with input shape (1, 3, 2048, 2048) BCHW and output shape(s) (1, 13, 86016) (6.5 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.28...
ONNX: export success ✅ 2.8s, saved as nn/origin/pose/yolov8n/pretrained/weights/best.onnx (13.4 MB)

Export complete (3.7s)
Results saved to /efs/players/to122838/projects/lard-yolov8/nn/origin/pose/yolov8n/pretrained/weights
Predict:         yolo predict task=pose model=nn/origin/pose/yolov8n/pretrained/wei

### From scratch

In [23]:
train("pose", "yolov8n-pose.yaml", batch_size=16, imgsz=2048, epochs=100, workers=8, patience=10, pretrained=False, optimizer="AdamW", lr0=0.001, cos_lr=True)

yolo pose train data=/aip/datasets/deel/processed/origin/pose/dataset.yaml model=yolov8n-pose.yaml batch=128 device=0,1,2,3,4,5,6,7 imgsz=2048 epochs=100 workers=8 patience=10 pretrained=False optimizer=AdamW lr0=0.001 cos_lr=True
Start new screen session ...


**Copy paste the run directory (ie `./runs/pose/train`) as `./nn/origin/pose/yolov8n/scratch`.**

In [11]:
model_filepath = Path("./nn/origin/pose/yolov8n/scratch/weights/best.pt")
assert model_filepath.exists()
info(model_filepath)

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Param %              Kernel Shape         Mult-Adds            Trainable
PoseModel (PoseModel)                              [1, 3, 2048, 2048]   [1, 13, 86016]       --                        --              --                   --                   False
├─Sequential (model)                               --                   --                   --                   -35.75%              --                   --                   False
│    └─Conv (0)                                    [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  --                        --              --                   --                   False
│    │    └─Conv2d (conv)                          [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  (432)                  0.01%              [3, 3]               452,984,832          False
│    │    └─BatchNorm2d (bn)                       [1, 16, 1024, 1024]  [1, 16, 1

In [15]:
validate("pose", model_filepath.as_posix(), screen=False, batch_size=8, imgsz=2048)

yolo pose val data=/aip/datasets/deel/processed/origin/pose/dataset.yaml model=nn/origin/pose/yolov8n/scratch/weights/best.pt batch=64 device=0,1,2,3,4,5,6,7 plots=True save_json=True imgsz=2048
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                                      CUDA:7 (NVIDIA A10G, 22564MiB)
YOLOv8n-pose summary (fused): 187 layers, 3077771 parameters, 0 gradients, 8.3 GFLOPs
val: Scanning /aip/datasets/deel/process

In [16]:
predict("pose", model_filepath.as_posix(), imgsz=2048)

yolo pose predict data=/aip/datasets/deel/processed/origin/pose/dataset.yaml model=nn/origin/pose/yolov8n/scratch/weights/best.pt device=0,1,2,3,4,5,6,7 source=./samples/LFMN_rjWG3WRM5yk_42.png save=True hide_labels=True imgsz=2048
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                             

Export in ONNX:

In [17]:
export_onnx(model_filepath.as_posix())

yolo export model=nn/origin/pose/yolov8n/scratch/weights/best.pt format=onnx simplify=True
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CPU
YOLOv8n-pose summary (fused): 187 layers, 3077771 parameters, 0 gradients, 8.3 GFLOPs

PyTorch: starting from nn/origin/pose/yolov8n/scratch/weights/best.pt with input shape (1, 3, 2048, 2048) BCHW and output shape(s) (1, 13, 86016) (6.5 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.28...
ONNX: export success ✅ 2.6s, saved as nn/origin/pose/yolov8n/scratch/weights/best.onnx (13.4 MB)

Export complete (3.5s)
Results saved to /efs/players/to122838/projects/lard-yolov8/nn/origin/pose/yolov8n/scratch/weights
Predict:         yolo predict task=pose model=nn/origin/pose/yolov8n/scratch/weights/best.onnx 

----

# Detect

## yolov8n

### From pretrained

In [30]:
train("detect", "yolov8n.pt", batch_size=16, imgsz=2048, epochs=100, workers=8, patience=10, pretrained=True, optimizer="AdamW", lr0=0.001, cos_lr=True)

yolo detect train data=/aip/datasets/deel/processed/origin/detect/dataset.yaml model=yolov8n.pt batch=128 device=0,1,2,3,4,5,6,7 imgsz=2048 epochs=100 workers=8 patience=10 pretrained=True optimizer=AdamW lr0=0.001 cos_lr=True
Start new screen session ...


**Copy paste the run directory (ie `./runs/detect/train`) as `./nn/origin/detect/yolov8n/pretrained`.**

In [31]:
model_filepath = Path("./nn/origin/detect/yolov8n/pretrained/weights/best.pt")
assert model_filepath.exists()
info(model_filepath)

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Param %              Kernel Shape         Mult-Adds            Trainable
DetectionModel (DetectionModel)                    [1, 3, 2048, 2048]   [1, 5, 86016]        --                        --              --                   --                   False
├─Sequential (model)                               --                   --                   --                   -36.56%              --                   --                   False
│    └─Conv (0)                                    [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  --                        --              --                   --                   False
│    │    └─Conv2d (conv)                          [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  (432)                  0.01%              [3, 3]               452,984,832          False
│    │    └─BatchNorm2d (bn)                       [1, 16, 1024, 1024]  [1, 16, 1

In [32]:
validate("detect", model_filepath.as_posix(), screen=False, batch_size=8, imgsz=2048)

yolo detect val data=/aip/datasets/deel/processed/origin/detect/dataset.yaml model=nn/origin/detect/yolov8n/pretrained/weights/best.pt batch=64 device=0,1,2,3,4,5,6,7 plots=True imgsz=2048
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                                      CUDA:7 (NVIDIA A10G, 22564MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /aip/datasets/deel/processed/origin/det

In [33]:
predict("detect", model_filepath.as_posix(), imgsz=2048)

yolo detect predict data=/aip/datasets/deel/processed/origin/detect/dataset.yaml model=nn/origin/detect/yolov8n/pretrained/weights/best.pt device=0,1,2,3,4,5,6,7 source=./samples/LFMN_rjWG3WRM5yk_42.png save=True hide_labels=True imgsz=2048
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                    

Export in ONNX:

In [34]:
export_onnx(model_filepath.as_posix())

yolo export model=nn/origin/detect/yolov8n/pretrained/weights/best.pt format=onnx simplify=True
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from nn/origin/detect/yolov8n/pretrained/weights/best.pt with input shape (1, 3, 2048, 2048) BCHW and output shape(s) (1, 5, 86016) (6.4 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.28...
ONNX: export success ✅ 2.2s, saved as nn/origin/detect/yolov8n/pretrained/weights/best.onnx (12.5 MB)

Export complete (3.1s)
Results saved to /efs/players/to122838/projects/lard-yolov8/nn/origin/detect/yolov8n/pretrained/weights
Predict:         yolo predict task=detect model=nn/origin/detect/yolov8n/pretrained

### From scratch

In [25]:
train("detect", "yolov8n.yaml", batch_size=16, imgsz=2048, epochs=100, workers=8, patience=10, pretrained=False, optimizer="AdamW", lr0=0.001, cos_lr=True)

yolo detect train data=/aip/datasets/deel/processed/origin/detect/dataset.yaml model=yolov8n.yaml batch=128 device=0,1,2,3,4,5,6,7 imgsz=2048 epochs=100 workers=8 patience=10 pretrained=False optimizer=AdamW lr0=0.001 cos_lr=True
Start new screen session ...


**Copy paste the run directory (ie `./runs/detect/train`) as `./nn/origin/detect/yolov8n/scratch`.**

In [12]:
model_filepath = Path("./nn/origin/detect/yolov8n/scratch/weights/best.pt")
assert model_filepath.exists()
info(model_filepath)

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Param %              Kernel Shape         Mult-Adds            Trainable
DetectionModel (DetectionModel)                    [1, 3, 2048, 2048]   [1, 5, 86016]        --                        --              --                   --                   False
├─Sequential (model)                               --                   --                   --                   -36.56%              --                   --                   False
│    └─Conv (0)                                    [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  --                        --              --                   --                   False
│    │    └─Conv2d (conv)                          [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  (432)                  0.01%              [3, 3]               452,984,832          False
│    │    └─BatchNorm2d (bn)                       [1, 16, 1024, 1024]  [1, 16, 1

In [23]:
validate("detect", model_filepath.as_posix(), screen=False, batch_size=8, pretrained=False, imgsz=2048)

yolo detect val data=/aip/datasets/deel/processed/origin/detect/dataset.yaml model=nn/origin/detect/yolov8n/scratch/weights/best.pt batch=64 device=0,1,2,3,4,5,6,7 plots=True save_json=True imgsz=2048
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                                      CUDA:7 (NVIDIA A10G, 22564MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /aip/datasets/deel/proces

In [24]:
predict("detect", model_filepath.as_posix(), imgsz=2048)

yolo detect predict data=/aip/datasets/deel/processed/origin/detect/dataset.yaml model=nn/origin/detect/yolov8n/scratch/weights/best.pt device=0,1,2,3,4,5,6,7 source=./samples/LFMN_rjWG3WRM5yk_42.png save=True hide_labels=True imgsz=2048
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                       

Export in ONNX:

In [25]:
export_onnx(model_filepath.as_posix())

yolo export model=nn/origin/detect/yolov8n/scratch/weights/best.pt format=onnx simplify=True
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CPU
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from nn/origin/detect/yolov8n/scratch/weights/best.pt with input shape (1, 3, 2048, 2048) BCHW and output shape(s) (1, 5, 86016) (6.4 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.28...
ONNX: export success ✅ 2.1s, saved as nn/origin/detect/yolov8n/scratch/weights/best.onnx (12.5 MB)

Export complete (3.0s)
Results saved to /efs/players/to122838/projects/lard-yolov8/nn/origin/detect/yolov8n/scratch/weights
Predict:         yolo predict task=detect model=nn/origin/detect/yolov8n/scratch/weights/best

# Segment

## yolov8n

### From pretrained

In [10]:
train("segment", "yolov8n-seg.pt", batch_size=14, imgsz=2048, epochs=100, workers=8, patience=10, pretrained=True, optimizer="AdamW", lr0=0.001, cos_lr=True)

yolo segment train data=/aip/datasets/deel/processed/origin/segment/dataset.yaml model=yolov8n-seg.pt batch=112 device=0,1,2,3,4,5,6,7 imgsz=2048 epochs=100 workers=8 patience=10 pretrained=True optimizer=AdamW lr0=0.001 cos_lr=True
Start new screen session ...


**Copy paste the run directory (ie `./runs/segment/train`) as `./nn/origin/segment/yolov8n/pretrained`.**

In [11]:
model_filepath = Path("./nn/origin/segment/yolov8n/pretrained/weights/best.pt")
assert model_filepath.exists()
info(model_filepath)

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Param %              Kernel Shape         Mult-Adds            Trainable
SegmentationModel (SegmentationModel)              [1, 3, 2048, 2048]   [1, 37, 86016]       --                        --              --                   --                   False
├─Sequential (model)                               --                   --                   --                   -34.09%              --                   --                   False
│    └─Conv (0)                                    [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  --                        --              --                   --                   False
│    │    └─Conv2d (conv)                          [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  (432)                  0.01%              [3, 3]               452,984,832          False
│    │    └─BatchNorm2d (bn)                       [1, 16, 1024, 1024]  [1, 16, 1

In [12]:
validate("segment", model_filepath.as_posix(), screen=False, batch_size=2, imgsz=2048)

yolo segment val data=/aip/datasets/deel/processed/origin/segment/dataset.yaml model=nn/origin/segment/yolov8n/pretrained/weights/best.pt batch=16 device=0,1,2,3,4,5,6,7 plots=True imgsz=2048
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                                      CUDA:7 (NVIDIA A10G, 22564MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs
val: Scanning /aip/datasets/deel/processed/

In [13]:
predict("segment", model_filepath.as_posix(), imgsz=2048)

yolo segment predict data=/aip/datasets/deel/processed/origin/segment/dataset.yaml model=nn/origin/segment/yolov8n/pretrained/weights/best.pt device=0,1,2,3,4,5,6,7 source=./samples/LFMN_rjWG3WRM5yk_42.png save=True hide_labels=True imgsz=2048
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                 

Export in ONNX:

In [14]:
export_onnx(model_filepath.as_posix())

yolo export model=nn/origin/segment/yolov8n/pretrained/weights/best.pt format=onnx simplify=True
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CPU
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from nn/origin/segment/yolov8n/pretrained/weights/best.pt with input shape (1, 3, 2048, 2048) BCHW and output shape(s) ((1, 37, 86016), (1, 32, 512, 512)) (6.9 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.28...
ONNX: export success ✅ 2.4s, saved as nn/origin/segment/yolov8n/pretrained/weights/best.onnx (13.5 MB)

Export complete (3.5s)
Results saved to /efs/players/to122838/projects/lard-yolov8/nn/origin/segment/yolov8n/pretrained/weights
Predict:         yolo predict task=segment model=n

### From scratch

In [20]:
train("segment", "yolov8n-seg.yaml", batch_size=14, imgsz=2048, epochs=100, workers=8, patience=10, pretrained=False, optimizer="AdamW", lr0=0.001, cos_lr=True)

yolo segment train data=/aip/datasets/deel/processed/origin/segment/dataset.yaml model=yolov8n-seg.yaml batch=112 device=0,1,2,3,4,5,6,7 imgsz=2048 epochs=100 workers=8 patience=10 pretrained=False optimizer=AdamW lr0=0.001 cos_lr=True
Start new screen session ...


**Copy paste the run directory (ie `./runs/segment/train`) as `./nn/origin/segment/yolov8n/scratch`.**

In [10]:
model_filepath = Path("./nn/origin/segment/yolov8n/scratch/weights/best.pt")
assert model_filepath.exists()
info(model_filepath)

/home/aws_install/miniconda3/envs/yolov8/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/aws_install/miniconda3/envs/yolov8/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type (var_name))                            Input Shape          Output Shape         Param #              Param %              Kernel Shape         Mult-Adds            Trainable
SegmentationModel (SegmentationModel)              [1, 3, 2048, 2048]   [1, 37, 86016]       --                        --              --                   --                   False
├─Sequential (model)                               --                   --                   --                   -34.09%              --                   --                   False
│    └─Conv (0)                                    [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  --                        --              --                   --                   False
│    │    └─Conv2d (conv)                          [1, 3, 2048, 2048]   [1, 16, 1024, 1024]  (432)                  0.01%              [3, 3]               452,984,832          False
│    │    └─BatchNorm2d (bn)                       [1, 16, 1024, 1024]  [1, 16, 1

In [41]:
validate("segment", model_filepath.as_posix(), screen=False, batch_size=2, imgsz=2048)

yolo segment val data=/aip/datasets/deel/processed/origin/segment/dataset.yaml model=nn/origin/segment/yolov8n/scratch/weights/best.pt batch=16 device=0,1,2,3,4,5,6,7 plots=True imgsz=2048
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                                      CUDA:7 (NVIDIA A10G, 22564MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs
val: Scanning /aip/datasets/deel/processed/ori

In [32]:
predict("segment", model_filepath.as_posix(), imgsz=2048)

yolo segment predict data=/aip/datasets/deel/processed/origin/segment/dataset.yaml model=nn/origin/segment/yolov8n/scratch/weights/best.pt device=0,1,2,3,4,5,6,7 source=./samples/LFMN_rjWG3WRM5yk_42.png save=True hide_labels=True imgsz=2048
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA A10G, 22564MiB)
                                                      CUDA:1 (NVIDIA A10G, 22564MiB)
                                                      CUDA:2 (NVIDIA A10G, 22564MiB)
                                                      CUDA:3 (NVIDIA A10G, 22564MiB)
                                                      CUDA:4 (NVIDIA A10G, 22564MiB)
                                                      CUDA:5 (NVIDIA A10G, 22564MiB)
                                                      CUDA:6 (NVIDIA A10G, 22564MiB)
                                    

Export in ONNX:

In [33]:
export_onnx(model_filepath.as_posix())

yolo export model=nn/origin/segment/yolov8n/scratch/weights/best.pt format=onnx simplify=True
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0 CPU
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from nn/origin/segment/yolov8n/scratch/weights/best.pt with input shape (1, 3, 2048, 2048) BCHW and output shape(s) ((1, 37, 86016), (1, 32, 512, 512)) (6.9 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.28...
ONNX: export success ✅ 2.4s, saved as nn/origin/segment/yolov8n/scratch/weights/best.onnx (13.5 MB)

Export complete (3.5s)
Results saved to /efs/players/to122838/projects/lard-yolov8/nn/origin/segment/yolov8n/scratch/weights
Predict:         yolo predict task=segment model=nn/origin/seg

---

# Results

In [13]:
from markdownTable import markdownTable

results = [
    {
        "task": "pose",
        "mult-adds<br>(GFLops)": 42.66,
        "weights": "pretrained",
        "mAP50<br>BBOX": 0.993,
        "mAP50:95<br>BBOX": 0.9,
        "mAP50<br>POSE": 0.985,
        "mAP50:95<br>POSE": 0.953,
    },
    {
        "task": "pose",
        "mult-adds<br>(GFLops)": 42.66,
        "weights": "scratch",
        "mAP50<br>BBOX": 0.981,
        "mAP50:95<br>BBOX": 0.852,
        "mAP50<br>POSE": 0.972,
        "mAP50:95<br>POSE": 0.912,
    }
]
print(markdownTable(results).setParams(row_sep = 'markdown').getMarkdown())

results = [
    {
        "task": "detect",
        "mult-adds<br>(GFLops)": 41.39,
        "weights": "pretrained",
        "mAP50<br>BBOX": 0.995,
        "mAP50:95<br>BBOX": 0.905,
    },
    {
        "task": "detect",
        "mult-adds<br>(GFLops)": 41.39,
        "weights": "scratch",
        "mAP50<br>BBOX": 0.989,
        "mAP50:95<br>BBOX": 0.871,
    }
]
print(markdownTable(results).setParams(row_sep = 'markdown').getMarkdown())

results = [
    {
        "task": "segment",
        "mult-adds<br>(GFLops)": 61.31,
        "weights": "pretrained",
        "mAP50<br>BBOX": 0.98,
        "mAP50:95<br>BBOX": 0.869,
        "mAP50<br>MASK": 0.974,
        "mAP50:95<br>MASK": 0.758,
    },
    {
        "task": "segment",
        "mult-adds<br>(GFLops)": 61.31,
        "weights": "scratch",
        "mAP50<br>BBOX": 0.989,
        "mAP50:95<br>BBOX": 0.871,
        "mAP50<br>MASK": 0.967,
        "mAP50:95<br>MASK": 0.732,
    }
]
print(markdownTable(results).setParams(row_sep = 'markdown').getMarkdown())

```|task|mult-adds<br>(GFLops)|  weights |mAP50<br>BBOX|mAP50:95<br>BBOX|mAP50<br>POSE|mAP50:95<br>POSE|
|----|---------------------|----------|-------------|----------------|-------------|----------------|
|pose|        42.66        |pretrained|     0.99    |       0.9      |     0.98    |      0.95      |
|pose|        42.66        |  scratch |     0.98    |      0.85      |     0.97    |      0.91      |```
```| task |mult-adds<br>(GFLops)|  weights |mAP50<br>BBOX|mAP50:95<br>BBOX|
|------|---------------------|----------|-------------|----------------|
|detect|        41.39        |pretrained|     0.99    |      0.91      |
|detect|        41.39        |  scratch |     0.99    |      0.87      |```
```|  task |mult-adds<br>(GFLops)|  weights |mAP50<br>BBOX|mAP50:95<br>BBOX|mAP50<br>MASK|mAP50:95<br>MASK|
|-------|---------------------|----------|-------------|----------------|-------------|----------------|
|segment|        61.31        |pretrained|     0.98    |      0.87      |  